In [1]:
# ===============================================================
# NOTEBOOK UTAMA UNTUK MENJALANKAN PIPELINE MODULAR
# ===============================================================

# 1. Impor semua library yang dibutuhkan
import os
import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Tuner, Trainer, Evaluator, Pusher
from tfx.proto import trainer_pb2, pusher_pb2, example_gen_pb2
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
import tensorflow_model_analysis as tfma

2025-07-08 14:34:38.899649: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-08 14:34:39.070509: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-08 14:34:38.830995: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-08 14:34:38.831566: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-08 14:34:38.958682: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
# 2. Konfigurasi Path dan Variabel
# --- GANTI DENGAN USERNAME ANDA ---
USERNAME = "raffihakim"
# -----------------------------------

# Nama pipeline baru untuk memastikan tidak ada cache lama
PIPELINE_NAME = "student-habits-modular-pipeline"
PIPELINE_ROOT = os.path.join(f'{USERNAME}-pipeline')
DATA_ROOT = "data"
SERVING_MODEL_DIR = os.path.join(PIPELINE_ROOT, "serving_model", PIPELINE_NAME)

# --- Path ke file-file logika Anda ---
TRANSFORM_MODULE_FILE = os.path.join("pipeline", "student_transform.py")
TUNER_MODULE_FILE = os.path.join("pipeline", "student_tuner.py")
TRAINER_MODULE_FILE = os.path.join("pipeline", "student_trainer.py")
# ------------------------------------

In [3]:
# 4. Inisialisasi Ulang InteractiveContext
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)
print(f"✅ Konteks TFX baru untuk '{PIPELINE_NAME}' berhasil diinisialisasi.")

✅ Konteks TFX baru untuk 'student-habits-modular-pipeline' berhasil diinisialisasi.


In [4]:
# ===============================================================
# 5. Jalankan Komponen Pipeline Secara Berurutan
# ===============================================================

# --- Jalankan setiap blok kode berikut dalam sel terpisah ---

# 1. ExampleGen
output_config = example_gen_pb2.Output(split_config=example_gen_pb2.SplitConfig(splits=[
    example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
    example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
]))
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output_config)
context.run(example_gen)
print("✅ 1/9 ExampleGen Selesai")

✅ 1/9 ExampleGen Selesai


In [5]:
# 2. StatisticsGen
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)
print("✅ 2/9 StatisticsGen Selesai")

✅ 2/9 StatisticsGen Selesai


In [6]:
# 3. SchemaGen
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)
print("✅ 3/9 SchemaGen Selesai")


✅ 3/9 SchemaGen Selesai


In [7]:
# 4. ExampleValidator
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'], schema=schema_gen.outputs['schema'])
context.run(example_validator)
print("✅ 4/9 ExampleValidator Selesai")

✅ 4/9 ExampleValidator Selesai


In [8]:
# 5. Transform
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=TRANSFORM_MODULE_FILE
)
context.run(transform)
print("✅ 5/9 Transform Selesai")

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying module.py -> build/lib
copying student_trainer.py -> build/lib
copying student_transform.py -> build/lib
copying student_tuner.py -> build/lib
installing to /tmp/tmps7l3tgqr
running install
running install_lib
copying build/lib/student_transform.py -> /tmp/tmps7l3tgqr
copying build/lib/module.py -> /tmp/tmps7l3tgqr
copying build/lib/student_trainer.py -> /tmp/tmps7l3tgqr
copying build/lib/student_tuner.py -> /tmp/tmps7l3tgqr
running install_egg_info


/mnt/d/proyek1/mlops/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmps7l3tgqr/tfx_user_code_Transform-0.0+001467eb49da7fd68fe1c87501132413029ed8032fb55aaf26e418633ecfce9d.egg-info
running install_scripts
creating /tmp/tmps7l3tgqr/tfx_user_code_Transform-0.0+001467eb49da7fd68fe1c87501132413029ed8032fb55aaf26e418633ecfce9d.dist-info/WHEEL
creating '/tmp/tmp8wvpr843/tfx_user_code_Transform-0.0+001467eb49da7fd68fe1c87501132413029ed8032fb55aaf26e418633ecfce9d-py3-none-any.whl' and adding '/tmp/tmps7l3tgqr' to it
adding 'module.py'
adding 'stude

2025-07-08 14:36:46.519629: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-08 14:36:46.521921: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


INFO:tensorflow:Assets written to: raffihakim-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/caeb889c88b442c189d027d76060f342/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


✅ 5/9 Transform Selesai


In [9]:
# 6. Tuner
tuner = Tuner(
    module_file=TUNER_MODULE_FILE,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=500)
)
context.run(tuner)
print("✅ 6/9 Tuner Selesai")

Trial 10 Complete [00h 00m 04s]
val_root_mean_squared_error: 7.156743049621582

Best val_root_mean_squared_error So Far: 4.656599044799805
Total elapsed time: 00h 00m 43s
Results summary
Results in raffihakim-pipeline/.temp/6/student_performance_tuning
Showing 10 best trials
Objective(name="val_root_mean_squared_error", direction="min")

Trial 02 summary
Hyperparameters:
num_layers: 2
units_0: 160
dropout: 0.2
learning_rate: 0.01
units_1: 192
units_2: 192
Score: 4.656599044799805

Trial 06 summary
Hyperparameters:
num_layers: 2
units_0: 64
dropout: 0.5
learning_rate: 0.001
units_1: 224
units_2: 128
Score: 4.7443060874938965

Trial 03 summary
Hyperparameters:
num_layers: 2
units_0: 192
dropout: 0.4
learning_rate: 0.001
units_1: 192
units_2: 192
Score: 4.843631267547607

Trial 07 summary
Hyperparameters:
num_layers: 1
units_0: 32
dropout: 0.1
learning_rate: 0.01
units_1: 224
units_2: 256
Score: 4.921535968780518

Trial 04 summary
Hyperparameters:
num_layers: 1
units_0: 256
dropout: 0.4
l

In [10]:
# 7. Trainer
trainer = Trainer(
    module_file=TRAINER_MODULE_FILE,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000)
)
context.run(trainer)
print("✅ 7/9 Trainer Selesai")

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying module.py -> build/lib
copying student_trainer.py -> build/lib
copying student_transform.py -> build/lib
copying student_tuner.py -> build/lib
installing to /tmp/tmp4dkeu81z
running install
running install_lib
copying build/lib/student_transform.py -> /tmp/tmp4dkeu81z
copying build/lib/module.py -> /tmp/tmp4dkeu81z
copying build/lib/student_trainer.py -> /tmp/tmp4dkeu81z
copying build/lib/student_tuner.py -> /tmp/tmp4dkeu81z
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.

/mnt/d/proyek1/mlops/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing ./raffihakim-pipeline/_wheels/tfx_user_code_Trainer-0.0+001467eb49da7fd68fe1c87501132413029ed8032fb55aaf26e418633ecfce9d-py3-none-any.whl
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 age_xf (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 study_hours_per_day_xf (In  [(None, 1)]                  0         []                            
 putLayer)                                                                                        
                                                                                                  
 social_media_hours_xf (Inp  [(None, 1)]                  0         []                            
 utLayer)                                 

INFO:tensorflow:Assets written to: raffihakim-pipeline/Trainer/model/7/Format-Serving/assets


✅ 7/9 Trainer Selesai


In [15]:
# 8. Resolver
blessed_model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id('latest_blessed_model_resolver')
context.run(blessed_model_resolver)
print("✅ 8/9 Resolver Selesai")

✅ 8/9 Resolver Selesai


In [16]:
# 9. Evaluator & Pusher
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='exam_score_xf')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[tfma.MetricsSpec(metrics=[tfma.MetricConfig(class_name='RootMeanSquaredError')])]
)
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=blessed_model_resolver.outputs.get('model'),
    eval_config=eval_config,
    transform_graph=transform.outputs['transform_graph']
)
context.run(evaluator, enable_cache=False)
print("✅ 9/9 Evaluator Selesai")

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(filesystem=pusher_pb2.PushDestination.Filesystem(base_directory=SERVING_MODEL_DIR))
)
context.run(pusher)
print("🚀🚀🚀 Pusher Selesai, Pipeline Lengkap! 🚀🚀🚀")


TypeError: Evaluator.__init__() got an unexpected keyword argument 'transform_graph'

In [13]:
!saved_model_cli show

2025-07-08 14:51:37.624448: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-08 14:51:37.634380: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-08 14:51:37.737553: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-08 14:51:37.737707: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-08 14:51:37.750224: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to